In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import drive
# Monta el Google Drive
drive.mount('/content/drive')

In [ ]:
df_filtrado2= pd.read_csv('/content/drive/MyDrive/Seminario de Titulo - Prof Roberto Muñoz/Maximiliano -  Generación y Análisis de Secuencias de Actividad/logs/df_semestre_full_2024_S1_test.csv')

#Creación variables

In [ ]:

# Asegurarse de que 'Fecha' está en formato datetime
df_filtrado2['Fecha'] = pd.to_datetime(df_filtrado2['Fecha'], errors='coerce')
# Eliminar filas con fechas nulas después de la conversión
#df_filtrado.dropna(subset=['Fecha'], inplace=True)


# Usamos la fecha máxima en todo el dataframe como proxy del fin del semestre
ultimo_log_semestre = df_filtrado2['Fecha'].max()

# Función para calcular la máxima racha de días consecutivos con acceso y la máxima brecha INTERNA
# Nota: Esta función ahora solo calcula la racha de acceso y la brecha ENTRE accesos
def calcular_rachas_internas(fechas_sorted):
    # Asegurarse de que las fechas estén ordenadas, aunque transform usualmente mantiene el orden del grupo
    fechas = fechas_sorted.sort_values()

    if fechas.empty:
        return 0, 0 # No hay fechas, 0 acceso, 0 sin acceso

    # Inicializar para la racha de acceso
    max_access = access_count = 1

    # Inicializar para la brecha sin acceso (interna)
    max_internal_no_access = 0 # Máxima brecha *entre* accesos

    if len(fechas) > 1:
        prev_date = fechas.iloc[0]
        for fecha in fechas.iloc[1:]:
            diff = (fecha - prev_date).days

            if diff == 1:  # Día consecutivo con acceso
                access_count += 1
            else:  # Hueco de días sin acceso entre accesos
                internal_no_access_gap = diff - 1
                max_internal_no_access = max(max_internal_no_access, internal_no_access_gap)
                access_count = 1  # Reiniciar contador de acceso

            max_access = max(max_access, access_count)
            prev_date = fecha

    # La función ahora devuelve la racha de acceso y la MÁXIMA BRECHA INTERNA
    return max_access, max_internal_no_access

# 1. Calcular la máxima racha de días CON acceso (usando la función tal cual para la primera parte del retorno)
# Esta parte sigue siendo correcta
df_filtrado2['max_days_with_access'] = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'] \
    .transform(lambda fechas: calcular_rachas_internas(fechas)[0])

# 2. Calcular la máxima brecha INTERNA de días SIN acceso (usando la segunda parte del retorno)
# Esta será la base para el cálculo final
df_filtrado2['max_internal_days_without_access'] = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'] \
    .transform(lambda fechas: calcular_rachas_internas(fechas)[1])

# 3. Calcular la brecha FINAL: desde el último acceso hasta el fin del semestre proxy
# Primero, encontrar la última fecha de acceso para cada usuario
last_access_dates = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'].transform('max')

# Calcular la diferencia en días entre el último acceso y la fecha fin del semestre proxy
time_since_last_access = (ultimo_log_semestre - last_access_dates).dt.days

# La brecha final es esta diferencia menos 1, asegurando que no sea negativa
# Por ejemplo, si el último acceso fue HOY (diff 0), la brecha final es 0.
# Si el último acceso fue AYER (diff 1), la brecha final es 0.
# Si el último acceso fue hace 2 días (diff 2), la brecha final es 1 día sin acceso.
final_inactivity_gap = np.maximum(0, time_since_last_access - 1)


# 4. El 'max_days_without_access' final es el máximo entre la brecha interna y la brecha final
df_filtrado2['max_days_without_access'] = np.maximum(
    df_filtrado2['max_internal_days_without_access'],
    final_inactivity_gap
)

# Opcional: Eliminar la columna intermedia si ya no la necesitas
df_filtrado2.drop(columns=['max_internal_days_without_access'], inplace=True)




In [ ]:
df_filtrado2

first lag logs

In [ ]:
# Filtrar el primer log y el último log por 'Nombre completo del usuario'
first_last_logs = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'].agg(['first', 'last']).reset_index()

# Calcular la diferencia entre el primer y el último log
first_last_logs['first_last_log_diff'] = (first_last_logs['last'] - first_last_logs['first']).dt.days

# Si ya existe la columna 'first_last_log_diff' en base, eliminarla antes de hacer el merge
if 'first_last_log_diff' in df_filtrado2.columns:
    df_filtrado2 = df_filtrado.drop(columns=['first_last_log_diff'])  # Eliminar antes de mergear

# Mergear los resultados al dataframe original usando 'Nombre completo del usuario'
df_filtrado2 = df_filtrado2.merge(first_last_logs[['Nombre completo del usuario', 'first_last_log_diff']], on='Nombre completo del usuario', how='left')

# Verificar el resultado
print(df_filtrado2[['Nombre completo del usuario', 'Fecha', 'first_last_log_diff']].drop_duplicates().head(10))

In [ ]:
df_filtrado2

[logs - weeks_logs - Weekly_logs]

In [ ]:

# Asegurarse de que 'Fecha' esté en formato datetime
#df_filtrado2['Fecha'] = pd.to_datetime(df_filtrado['Fecha'], errors='coerce')

# 1. Calcular total de logs por alumno usando value_counts()
interacciones_por_alumno = df_filtrado2['Nombre completo del usuario'].value_counts()

# Asignar al dataframe original
df_filtrado2['logs'] = df_filtrado2['Nombre completo del usuario'].map(interacciones_por_alumno)

# 2. Calcular logs totales por semana para cada alumno
# Contamos las interacciones de cada alumno por semana
#df_filtrado2['week_logs'] = df_filtrado2.groupby(['Nombre completo del usuario', 'semana_semestre'])['semana_semestre'].transform('count')

# 3. Calcular promedio de logs diarios por alumno
# Contar días activos únicos por alumno
dias_activos = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'].transform('nunique')
# Calcular el promedio de logs diarios
df_filtrado2['daily_avg'] = df_filtrado2['logs'] / dias_activos

# 4. Calcular promedio de logs semanales por alumno
# Contar semanas activas únicas por alumno
semanas_activas = df_filtrado2.groupby('Nombre completo del usuario')['semana_semestre'].transform('nunique')
# Calcular el promedio de logs semanales
df_filtrado2['weekly_avg'] = df_filtrado2['logs'] / semanas_activas

# Verificar el resultado
print(df_filtrado2[['Nombre completo del usuario', 'Fecha', 'logs', 'week_logs', 'daily_avg', 'weekly_avg']].head())

days_with_logs_...


In [ ]:


# 1. Total de días únicos con logs por alumno
df_filtrado2['days_with_logs'] = df_filtrado2.groupby('Nombre completo del usuario')['Fecha'].transform('nunique')

# 2. Promedio de días activos por alumno (total días / semanas activas)
# Contar semanas activas únicas por alumno
semanas_activas = df_filtrado2.groupby('Nombre completo del usuario')['semana_semestre'].transform('nunique')
df_filtrado2['days_with_logs_avg'] = df_filtrado2['days_with_logs'] / semanas_activas

# 3. Días únicos con logs por semana para cada alumno
days_with_logs_per_week = (
    df_filtrado2.groupby(['Nombre completo del usuario', 'semana_semestre'])['Fecha']
    .transform('nunique')
)
df_filtrado2['days_with_logs_week'] = days_with_logs_per_week

# 4. Promedio de días únicos con logs por semana
df_filtrado2['days_with_logs_avg_week'] = df_filtrado2.groupby('Nombre completo del usuario')['days_with_logs_week'].transform('mean')

# Verificar el resultado
print(df_filtrado2[['Nombre completo del usuario', 'Fecha', 'days_with_logs', 'days_with_logs_avg', 'days_with_logs_week', 'days_with_logs_avg_week']].head())


*Clasificar* eventos

In [ ]:

# --- PASO 1: Función para Clasificar Eventos (Mejorada) ---
def clasificar_evento_mejorado(evento_original):
    evento = str(evento_original).lower() # Convertir a minúsculas y asegurar que sea string

    # REPORT: Eventos relacionados con la visualización de informes, notas o calificaciones.
    if 'informe de notas' in evento or \
       'informe usuario del curso visto' in evento or \
       'informe del usuario visualizado' in evento or \
       'informe de resumen de notas visto' in evento or \
       'usuario calificado' in evento or \
       'elemento de calificación actualizado' in evento:
        return 'report'

    # ACTIVITY: Acciones directas del estudiante sobre elementos de aprendizaje, entregas, cuestionarios, comentarios.
    elif 'intento enviado' in evento or \
         'entrega creada' in evento or \
         'se ha enviado una entrega' in evento or \
         'un fichero ha sido subido' in evento or \
         'ha comenzado el intento' in evento or \
         'envío actualizado.' in evento or \
         'envío eliminado' in evento or \
         'intento de cuestionario actualizado' in evento or \
         'intento del cuestionario revisado' in evento:
        return 'activity'
    elif 'formulario de entrega visto' in evento or \
         'se ha visualizado el estado de la entrega.' in evento or \
         'el estado de la entrega se ha actualizado.' in evento or \
         'borrar la confirmación de envío visualizada.' in evento or \
         'resumen del intento de cuestionario visualizado' in evento:
        return 'activity'
    elif 'intento de cuestionario visualizado' in evento: # Considerado 'activity' por la interacción.
        return 'activity'
    elif 'comentario creado' in evento or \
         'comentario eliminado' in evento:
        return 'activity'

    # CONTENT: Visualización de materiales del curso, recursos.
    elif 'curso visto' in evento or \
         'módulo de curso visto' in evento or \
         'instancia del módulo del curso visualizada' in evento or \
         'directorio descargado' in evento or \
         'sección del curso creada' in evento:
        return 'content'

    # SYSTEM: Interacciones con funcionalidades de la plataforma o acciones automáticas.
    elif 'intento de cuestionario guardado automáticamente' in evento or \
         'tour iniciado' in evento or \
         'tour terminado' in evento or \
         'tour reiniciado' in evento:
        return 'system'

    # OTHER: Acciones que no encajan claramente o son genéricas/administrativas.
    elif 'perfil de usuario visto' in evento:
        return 'other'
    elif 'elemento de calificación creado' in evento: # Raro, posible acción administrativa
        return 'other'
    else:
        # print(f"Evento no clasificado (categorizado como 'other'): {evento_original}") # Descomenta para depurar
        return 'other'

# --- PASO 3: Aplicar la Clasificación de Eventos ---

# This line is where the error happened because df_filtrado2 was a string.
# Assuming df_filtrado2 is now a DataFrame, this line should work.
df_filtrado2['categoria_evento'] = df_filtrado2['Nombre evento'].apply(clasificar_evento_mejorado)

for cat in ['activity', 'content', 'report', 'system', 'other']:
  # This line is also sensitive to df_filtrado2 being a DataFrame.
  # If df_filtrado2 is a string, this fails.
  df_filtrado2[cat + '_total'] = df_filtrado2.groupby('Nombre completo del usuario')['categoria_evento'] \
  .transform(lambda x: (x == cat).sum()) # Use 'cat' here as well
  print("Columnas _total calculadas.") # This print statement is still inside the loop

# Verificar los conteos de categoria_evento
print("\nConteo de eventos por 'categoria_evento' después de la clasificación:")
print(df_filtrado2['categoria_evento'].value_counts())

# Verificar algunas de las columnas _total
print("\nEjemplo de las primeras filas con columnas _total:")
columnas_a_ver = ['Nombre completo del usuario', 'Nombre evento', 'categoria_evento', 'activity_total', 'content_total', 'report_total', 'system_total']
# Asegurarse de que todas las columnas existen antes de intentar mostrarlas
columnas_existentes_para_ver = [col for col in columnas_a_ver if col in df_filtrado2.columns]
print(df_filtrado2[columnas_existentes_para_ver].head())

In [ ]:
df_filtrado2

{Activiy,content,other}_week

In [ ]:

# Calcular los conteos de eventos por categoría, semana y alumno
conteo_eventos_semana = df_filtrado2.groupby(['Nombre completo del usuario', 'semana_semestre', 'categoria_evento']).size().unstack(fill_value=0)

# Preparar los conteos semanales para el merge y renombrar columnas
conteo_eventos_semana = conteo_eventos_semana.reset_index()

# Añadir sufijo '_week' a las columnas de categoría
columnas_categorias = [col for col in conteo_eventos_semana.columns if col not in ['Nombre completo del usuario', 'semana_semestre']]
rename_dict = {col: f"{col}_week" for col in columnas_categorias}
conteo_eventos_semana.rename(columns=rename_dict, inplace=True)

# Mergear los conteos semanales de vuelta al dataframe original
df_filtrado2 = df_filtrado2.merge(conteo_eventos_semana,
                                on=['Nombre completo del usuario', 'semana_semestre'],
                                how='left')

# --- Opcional: Verificar el resultado ---
print("DataFrame con conteos semanales por categoría añadidos (ejemplo):")
columnas_a_mostrar = ['Nombre completo del usuario', 'semana_semestre'] + [f"{cat}_week" for cat in columnas_categorias]
print(df_filtrado2[columnas_a_mostrar].drop_duplicates().head())

In [ ]:
# 3. Calcular `total_weeks`: Total de semanas activas en el semestre
df_filtrado2['total_weeks'] = df_filtrado2.groupby('Nombre completo del usuario')['semana_semestre'].transform('nunique')

In [ ]:
# Select multiple columns by passing a list of column names
print(df_filtrado2[['Nombre completo del usuario', 'total_weeks']])

In [ ]:
# También puedes ver algunas filas donde no son cero:
print("\nEjemplos donde report_week2 > 0:")
print(df_filtrado2[df_filtrado2['report_week'] > 0][['Nombre completo del usuario', 'semana_semestre', 'report_week']].head())

print("\nEjemplos donde system_week2 > 0:")
print(df_filtrado2[df_filtrado2['system_week'] > 0][['Nombre completo del usuario', 'semana_semestre', 'system_week']].head())

In [ ]:
# Función para calcular la mayor racha de semanas consecutivas con actividad
def calcular_racha_semanas(weeks):
    weeks = sorted(set(weeks))  # Asegurar que las semanas sean únicas y estén ordenadas
    max_streak = streak = 1
    for i in range(1, len(weeks)):
        if weeks[i] == weeks[i-1] + 1:  # Si la semana es consecutiva a la anterior
            streak += 1
            max_streak = max(max_streak, streak)
        else:
            streak = 1  # Reiniciar la racha si las semanas no son consecutivas
    return max_streak

# Aplicar la función por cada alumno (usando 'Nombre completo del usuario')
rachas = df_filtrado2.groupby('Nombre completo del usuario')['semana_semestre'].apply(calcular_racha_semanas)

# Asignar el resultado al dataframe original
df_filtrado2['longest_streak'] = df_filtrado2['Nombre completo del usuario'].map(rachas)

# Verificar el resultado
print(df_filtrado2[['Nombre completo del usuario', 'longest_streak']].head())

In [ ]:
# Guardar el DataFrame en un archivo CSV
df_filtrado2.to_csv('/content/drive/MyDrive/Seminario de Titulo - Prof Roberto Muñoz/Maximiliano -  Generación y Análisis de Secuencias de Actividad/logs/.csv', index=False)

print("Archivo CSV guardado correctamente.")

Agregar Variables notas

In [ ]:
#Alumnos
# Verificar que la ruta al archivo es correcta



# Ruta del archivo
file_path = '/content/drive/MyDrive/Seminario de Titulo - Prof Roberto Muñoz/Maximiliano -  Generación y Análisis de Secuencias de Actividad/logs/Notas_2024_1_final.xlsx'

# Nombre de la hoja donde está la lista de alumnos
hoja_alumnos = "Notas"

import os
juh
# Verificar si la ruta existe
if os.path.exists(file_path):
    print("Archivo encontrado.")
    lista_notas_2024_1 = pd.read_excel(file_path, sheet_name=hoja_alumnos)
else:
    print("El archivo no fue encontrado. Verifica la ruta.")

In [ ]:
lista_notas['Nombre completo del usuario'] = (
    lista_notas['Nombre completo del usuario']
    .str.strip()             # Elimina espacios al inicio y al final
    .str.replace(r'\s+', ' ', regex=True)  # Sustituye múltiples espacios por uno solo
    .str.lower()             # Convierte a minúsculas
)

In [ ]:
# Eliminar espacios al principio y al final de la columna 'Nombre completo del usuario'
lista_notas['Nombre completo del usuario'] = lista_notas['Nombre completo del usuario'].str.strip()

# Verificar los primeros registros
print(lista_notas.head())


In [ ]:
# Redondear a 2 decimales (esto redondeará números como 1.925 a 1.93)
lista_notas['Final'] = lista_notas['Final'].apply(lambda x: round(x, 2))

lista_notas['aprobo_semestre_real'] = lista_notas['Final'].apply(lambda x: 1 if x >= 4.0 else 0)

In [ ]:
# Crear el promedio ponderado con las notas disponibles
lista_notas['promedio_ponderado'] = (lista_notas['Sumativa 1'] * 0.25) + (lista_notas['Control 1'] * 0.05) + \
                                  (lista_notas['Control 2'] * 0.05)


# Redondear a 2 decimales (esto redondeará números como 1.925 a 1.93)
lista_notas['promedio_ponderado'] = lista_notas['promedio_ponderado'].apply(lambda x: round(x, 2))

In [ ]:
lista_notas['proyeccion_nota_final'] = lista_notas['promedio_ponderado'] / 0.35

In [ ]:
df_filtrado2 = df_filtrado2.merge(lista_notas[['Nombre completo del usuario','Sumativa 1','Control 1','Control 2','promedio_ponderado','aprobo_semestre_real'],
                                      on='Nombre completo del usuario',
                                      how='left')

In [ ]:
df_filtrado2['aprobando'] = (
    (df_filtrado2['proyeccion_nota_final'] >= 4.0) &
    (df_filtrado2['logs'] >= 500)
).astype(int)


In [ ]:
df_2023_1['aprobo_semestre_real'] = df_2023_1['aprobo_semestre_real'].fillna(0)